In [13]:
%run function_dbs.py
%run hyperStack_basic.py
%matplotlib

Using matplotlib backend: MacOSX


In [3]:
# get link to data
pathData = '/Volumes/HIS-CAMEL/04measurementData/03_projects/06_pH+O2/20220718/image_scene/460nm/5ms/'
file_calib_overview = '202207_calibrationPoints.xlsx'

## collect information about calibration points

In [5]:
df_head = pd.read_excel(file_calib_overview)
df_head_ = df_head[df_head['LED'] == int(pathData.split('/')[-3].split('nm')[0])]
df_head = df_head_[df_head_['integration time (ms)'] == int(pathData.split('/')[-2].split('m')[0])]
df_head.head(5)

,pH,O2 (%O2),LED,integration time (ms),Unnamed: 4
0,6.0100,20.35,460,5,y
4,6.5050,20.42,460,5,y
8,5.4900,20.41,460,5,y
12,4.9900,20.36,460,5,y
16,3.9615,20.33,460,5,y


## inspect image cubes and extract spectral info

In [20]:
ls_files = list([f for f in glob(pathData + '/*.hdr') if 'corrected' in f])
print('number of collected files:', len(ls_files))

# --------------------------------
# get pH and O2 information from the excel
ls_pH = df_head['pH'].to_numpy()
ls_ph = list(dict.fromkeys(list([int(o) for o in ls_pH])))

ls_O2 = df_head['O2 (%O2)'].to_numpy()
ls_ox = list(dict.fromkeys(list([int(o) for o in ls_O2])))

# --------------------------------
# sort files according to oxygen / pH calibration points (prep for RoI selection)
ls_calib = list()
for f in ls_files:
    calib = f.split('pH')[-1].split('O2')[0].split('-')
    l = [np.float(c.replace(',', '.')) for c in calib]
    ls_calib.append(l)
    
df_files = pd.concat([pd.DataFrame(ls_files), pd.DataFrame(ls_calib, columns = ['pH', 'O2 (%O2)'])], axis=1)
df_files = df_files.sort_values(by=['pH', 'O2 (%O2)'])
df_files.head(5)

number of collected files: 40


,0,pH,O2 (%O2)
0,/Volumes/HIS-CAMEL/04measurementData/03_projec...,1.7,20.34
9,/Volumes/HIS-CAMEL/04measurementData/03_projec...,2.0,0.00
10,/Volumes/HIS-CAMEL/04measurementData/03_projec...,4.0,0.00
15,/Volumes/HIS-CAMEL/04measurementData/03_projec...,4.0,6.00
11,/Volumes/HIS-CAMEL/04measurementData/03_projec...,4.0,10.40


In [7]:
# collect overall cube-info
para = dict(map(lambda en: (tuple(df_files.loc[en+1][['pH', 'O2 (%O2)']].to_numpy()), load_cube(file_hdr=ls_files[en+1])), 
                range(len(df_files[0].to_numpy())-1)))

In [8]:
# get individual data
ls_wl = dict(map(lambda en: (en, list(para[en]['Wavelength'])), para.keys()))
data = dict(map(lambda en: (en, para[en]['cube'].open_memmap()), para.keys()))

In [9]:
# select sets 
ls_set1 = sorted(list(list(data.keys())[33:37]) + list(list(data.keys())[38:]) +  list(list(data.keys())[30:33]))
ls_set2 = sorted(list(list(data.keys())[25:30]))
ls_set3 = sorted(list(list(data.keys())[19:21]) + list(list(data.keys())[24:25]))
ls_set4 = sorted(list(list(data.keys())[21:22]) + list(list(data.keys())[23:24]))
ls_set5 = sorted(list(list(data.keys())[10:12]))
ls_set6 = sorted(list(list(data.keys())[18:19]) + list(list(data.keys())[22:23])) 
ls_set7 = sorted(list(list(data.keys())[7:10]))
ls_set8 = sorted(list(list(data.keys())[:7]))
ls_set9 = sorted(list(list(data.keys())[12:13]) + list(list(data.keys())[15:16]) + list(list(data.keys())[17:18]))
ls_set10 = sorted(list(list(data.keys())[13:15]))
ls_set11 = sorted(list(list(data.keys())[16:17]))
ls_set12 = sorted(list(list(data.keys())[37:38]))

ls_sets = [ls_set1, ls_set2, ls_set3, ls_set4, ls_set5, ls_set6, ls_set7, ls_set8, ls_set9, ls_set10, ls_set11, ls_set12]

In [10]:
# adjust RoI to secure you always use the correct area in the optode --> small area
RoI_optode = [[(1200, 600), (1300, 600), (1300, 700), (1200, 700)], # set-1
              [(1250, 660), (1317, 660), (1317, 740), (1250, 740)], # set-2
              [(1150, 690), (1300, 690), (1300, 760), (1150, 760)], # set-3
              [(1260, 710), (1290, 710), (1290, 740), (1260, 740)], # set-4
              [(1280, 650), (1360, 650), (1360, 730), (1280, 730)], # set-5
              [(1000, 610), (1175, 610), (1175, 680), (1000, 680)], # set-6
              [(1000, 620), (1050, 620), (1050, 740), (1000, 740)], # set-7
              [(1150, 600), (1350, 600), (1350, 750), (1150, 750)], # set-8
              [(1230, 600), (1420, 600), (1420, 750), (1230, 750)], # set-9
              [(1250, 550), (1350, 550), (1350, 750), (1250, 750)], # set-10
              [(1100, 510), (1350, 510), (1350, 750), (1100, 750)], # set-11
              [(1075, 680), (1275, 680), (1275, 800), (1075, 800)]] # set-12

# -----------------------------------
set_ = 1
for k, l in enumerate(ls_sets[set_]):
    fig, ax = plt.subplots()
    fig.canvas.set_window_title(df_files[0].to_numpy()[k].split('/')[-1].split('_corr')[0])
    
    if RoI_optode:
        df = data[l][:, :, 100][RoI_optode[set_][1][1]: RoI_optode[set_][2][1], RoI_optode[set_][0][0]: RoI_optode[set_][1][0]]
    else:
        df = data[l][:, :, 100]

    ax.imshow(df, vmin=0, vmax=0.65)
    plt.tight_layout(pad=1)
    plt.show()
    
    
    # save figure 
    # fig.savefig('Plot/RoI_optodes/' + str(i) + '_' + title + '.png')

In [11]:
ddata = dict()
for en, l in enumerate(ls_sets):  
    for li in l:
        # crop data frame
        df = data[li][:, :, :][RoI_optode[en][1][1]: RoI_optode[en][2][1], RoI_optode[en][0][0]: RoI_optode[en][1][0]]

        # collect pH and O2 information
        ph, ox = li[0], li[1]

        # store data in dictionary
        ddata[(ph, ox)] = df


In [14]:
# make sure the RoI have the same size
ls_imsize = [ddata[i].shape[0]*ddata[i].shape[1] for i in ddata.keys()]
indices = [i for i, x in enumerate(ls_imsize) if x ==  min(ls_imsize)]
smallestRoI = ddata[list(ddata.keys())[indices[0]]].shape[0]*ddata[list(ddata.keys())[indices[0]]].shape[1]

ls_reshape = list(map(lambda en: reshapeRoI(a=ddata[en].shape[0], b=ddata[en].shape[1], smallestRoI=smallestRoI), ddata.keys()))

ddata_reshaped = dict(map(lambda en: (en[1], ddata[en[1]][:ls_reshape[en[0]][0], :ls_reshape[en[0]][1], :]), enumerate(ddata.keys())))

# double-check size of RoI
[ddata_reshaped[en].shape[0]*ddata_reshaped[en].shape[1] for en in ddata_reshaped.keys()]

[900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 871,
 871,
 871,
 871,
 871,
 910,
 910,
 910,
 900,
 900,
 880,
 880,
 910,
 910,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 900,
 960,
 960]

In [15]:
# re-arange dataframe: calibration point, wavelength x width x height
ddata_swaped = dict(map(lambda c: (c, np.swapaxes(ddata_reshaped[c], 0, 2)), ddata_reshaped.keys()))

# outlier removal
data_start = dict(map(lambda en: (en, dict(map(lambda wl: (wl, _interquartOutlierRemove(ddata_swaped[en][wl])), 
                                               np.arange(len(ddata_swaped[en]))))), ddata_swaped.keys()))

###### split each calibration dataframe in an equal amount of chunks 
generates more data points -> avoid data augmentation but enables a more secure training data set

In [24]:
dic_sm = dict(map(lambda c: (c, split2chunks(df=data_start[c], ls_wl=ls_wl[c], ph=c[0], ox=c[1], n_pixels=5)), data_start.keys()))

# concat dataset to get a structured set
df_data = pd.concat(dic_sm)
df_data.index = np.arange(0, len(df_data.index))
df_data.shape

## save data

In [ ]:
# save the smaller RoI as csv
df_data.to_csv('TrainingData_absoluteIntensity.csv')